In [46]:
import os
import sys; sys.path.append("~/HHbbgg_ETH/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)

<module 'training_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/Training/python/training_utils.py'>

In [50]:
reload(utils)
ntuples = '20170620'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#pick only SM
signal = [s for s in files if "GluGluToHHTo2B2G_node_SM" in s]
diphotonJets = [s for s in files if "DiPhotonJets" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]
ggH = [s for s in files if "GluGluHToGG" in s]
vbf = [s for s in files if "VBFHToGG" in s]
VH = [s for s in files if "VHToGG" in s]
bbH = [s for s in files if "bbHToGG_M-125_4FS_yb2" in s] #is this correct?
ttH = [s for s in files if "ttHToGG" in s]

Data= [s for s in files if "DoubleEG" in s]

utils.IO.add_signal(ntuples,signal,1)
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-2)
utils.IO.add_background(ntuples,gJets_highPt,-2)
utils.IO.add_background(ntuples,ggH,-3)
utils.IO.add_background(ntuples,vbf,-4)
utils.IO.add_background(ntuples,VH,-5)
utils.IO.add_background(ntuples,bbH,-6)
utils.IO.add_background(ntuples,ttH,-7)

utils.IO.add_data(ntuples,Data,-10)


for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]
print "using data file: "+ utils.IO.dataName[0]

env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GluGluHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VBFHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8.

In [40]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#all variables
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M(),customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,DiJetDiPho_DR'.split(",")
#std variables
#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
print branch_names

import pandas as pd
import root_pandas as rpd

# no need to shuffle here, we just count events
utils.preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names,shuffle=False)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.set_variables(branch_names)

X_data,y_data,weights_data = utils.preprocessing.set_data("bbggSelectionTree",branch_names)
X_data,y_data,weights_data = utils.preprocessing.clean_signal_events_single_dataset(X_data,y_data,weights_data)

#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=utils.preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)

#np.savetxt('/tmp/micheli/xbkg.txt',X_bkg)
#utils.preprocessing.adjust_and_compress(utils.IO.background_df[0]).to_csv('/tmp/micheli/bkg.csv')


['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'DiJetDiPho_DR']


In [41]:
# load the model from disk
from sklearn.externals import joblib
#loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_stdVariables.pkl'))
loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH/Training/output_files/allMC_resWeighting_F.pkl'))

bkg_0 = X_bkg[y_bkg ==-1]
bkg_1 = X_bkg[y_bkg ==-2]
bkg_3 = X_bkg[y_bkg ==-3]
bkg_4 = X_bkg[y_bkg ==-4]
bkg_5 = X_bkg[y_bkg ==-5]
bkg_6 = X_bkg[y_bkg ==-6]
bkg_7 = X_bkg[y_bkg ==-7]



#bkg_1 = np.concatenate((X_total_train[y_total_train == -2],X_total_test[y_total_test == -2]))
#bkg_3 = np.concatenate((X_total_train[y_total_train == -3],X_total_test[y_total_test == -3]))
#bkg_4 = np.concatenate((X_total_train[y_total_train == -4],X_total_test[y_total_test == -4]))
#bkg_5 = np.concatenate((X_total_train[y_total_train == -5],X_total_test[y_total_test == -5]))
#bkg_6 = np.concatenate((X_total_train[y_total_train == -6],X_total_test[y_total_test == -6]))
#bkg_7 = np.concatenate((X_total_train[y_total_train == -7],X_total_test[y_total_test == -7]))


Y_pred_sig = loaded_model.predict_proba(X_sig)[:,2]
Y_pred_bkg_0 = loaded_model.predict_proba(bkg_0)[:,2]
Y_pred_bkg_1 = loaded_model.predict_proba(bkg_1)[:,2]
Y_pred_bkg_3 = loaded_model.predict_proba(bkg_3)[:,2]
Y_pred_bkg_4 = loaded_model.predict_proba(bkg_4)[:,2]
Y_pred_bkg_5 = loaded_model.predict_proba(bkg_5)[:,2]
Y_pred_bkg_6 = loaded_model.predict_proba(bkg_6)[:,2]
Y_pred_bkg_7 = loaded_model.predict_proba(bkg_7)[:,2]

Y_pred_data = loaded_model.predict_proba(X_data)[:,2]

print Y_pred_data 



[ 0.00224534  0.20553288  0.00474526 ...,  0.03072882  0.05181648
  0.6292057 ]


In [42]:
#define MVA cut and additional variables needed
MVA_Cut = 0.40
additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M(),isSignal,MX'.split(",")

In [53]:
#### here counting events only in dijet mass window######
#print utils.IO.signal_df[0]['proc']
#print utils.IO.signal_df[0].index
sig_count_df = rpd.read_root(utils.IO.signalName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(sig_count_df,utils.IO.sigProc[0],utils.IO.signalName[0])

#nTot is a multidim vector with all additional variables, dictVar is a dictionary associating a name of the variable
#to a position in the vector
nTot,dictVar = utils.postprocessing.stackFeatures(sig_count_df,additionalCut_names)
print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
print "Y_pred"
print Y_pred_sig.shape
nCleaned_MVA = nCleaned[np.where(Y_pred_sig>MVA_Cut)]
#apply massWindow +- 1.5
print nCleaned_MVA.shape
nCleaned_massWindowDown = utils.postprocessing.applyCut(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000)
nCleaned_massWindow = utils.postprocessing.applyCut(nCleaned_massWindowDown,dictVar['diphotonCandidate.M()'],126.50000,'smaller')
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print "signal events with 36 fb-1: "+str(nEvts)
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = utils.postprocessing.applyCut(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100)
nCleaned_mdijet = utils.postprocessing.applyCut(nCleaned_mdijet_down,dictVar['dijetCandidate.M()'],140,'smaller')
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,dictVar['diphotonCandidate.M()']]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,dictVar['dijetCandidate.M()']]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,dictVar['MX']]))
v.dtype = [('MX', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,dictVar['weight']]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,dictVar['diphotonCandidate.M()']]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,dictVar['dijetCandidate.M()']]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,dictVar['MX']]))
v.dtype = [('MX', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,dictVar['weight']]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_sig.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(21184, 5)
(18646, 5)
Y_pred
(18646,)
(16984, 5)
signal events with 36 fb-1: 9.50370146764
(8369, 5)
after cutting in dijet mass:6.06974400322


In [57]:
bkg_0_count_df = rpd.read_root(utils.IO.backgroundName[0],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_0_count_df,utils.IO.bkgProc[0],utils.IO.backgroundName[0])

nTot,dictVar = utils.postprocessing.stackFeatures(bkg_0_count_df,additionalCut_names)

print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_0>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = utils.postprocessing.applyCut(nCleaned_MVA,dictVar['diphotonCandidate.M()'],123.5000)
nCleaned_massWindow = utils.postprocessing.applyCut(nCleaned_massWindowDown,dictVar['diphotonCandidate.M()'],126.50000,'smaller')
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,dictVar['weight']])
print utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
print "with scale factor:"+str(nEvts*1.45) 
#Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = utils.postprocessing.applyCut(nCleaned_massWindow,dictVar['dijetCandidate.M()'],100)
nCleaned_mdijet = utils.postprocessing.applyCut(nCleaned_mdijet_down,dictVar['dijetCandidate.M()'],140,'smaller')
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,dictVar['weight']])
print "after cutting in dijet mass (with scale factor):"+str(nEvts_Cut*1.45)

from root_numpy import array2root
outTag = '20170711'
crazySF=1.45


processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,dictVar['diphotonCandidate.M()']]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,dictVar['dijetCandidate.M()']]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,dictVar['MX']]))
v.dtype = [('MX', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.multiply(np.asarray(nCleaned_mdijet[:,dictVar['weight']]),crazySF))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[0].split("/")[len(utils.IO.backgroundName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,dictVar['diphotonCandidate.M()']]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,dictVar['dijetCandidate.M()']]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,dictVar['MX']]))
v.dtype = [('MX', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.multiply(np.asarray(nCleaned[:,dictVar['weight']]),crazySF))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_0.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(2190915, 5)
(1900684, 5)
(225430, 5)
(10259, 5)
DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa events with 36 fb-1: 160.817862325
with scale factor:233.185900371
(3141, 5)
after cutting in dijet mass (with scale factor):69.6956103054


In [166]:
bkg_1_count_df = rpd.read_root(utils.IO.backgroundName[1],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_1_count_df,utils.IO.bkgProc[1],utils.IO.backgroundName[1])

M_bkg = np.asarray(bkg_1_count_df['diphotonCandidate.M()'][utils.IO.background_df[1].index])
crazySF_20=25
W_bkg = np.asarray(np.multiply(bkg_1_count_df['weight'][utils.IO.background_df[1].index],crazySF_20))
M_dijet_bkg = np.asarray(bkg_1_count_df['dijetCandidate.M()'][utils.IO.background_df[1].index])

nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))
print nTot.shape

bkg_2_count_df = rpd.read_root(utils.IO.backgroundName[2],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_2_count_df,utils.IO.bkgProc[2],utils.IO.backgroundName[2])

M_bkg = np.asarray(bkg_2_count_df['diphotonCandidate.M()'][utils.IO.background_df[2].index])
crazySF_40=3
W_bkg = np.asarray(np.multiply(bkg_2_count_df['weight'][utils.IO.background_df[2].index],crazySF_40))
M_dijet_bkg = np.asarray(bkg_2_count_df['dijetCandidate.M()'][utils.IO.background_df[2].index])

nTot_2 = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))

nTot_3 = np.concatenate((nTot,nTot_2))

print nTot_3.shape
#apply isSignal cleaning
nCleaned = nTot_3[np.where(nTot_3[:,1]!=0),:][0]
print "nCleaned"
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_1>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

#np.savetxt('/tmp/micheli/prova7.txt',nCleaned_mdijet)
#utils.preprocessing.adjust_and_compress(bkg_2_count_df).to_csv('/tmp/micheli/background_7_2.csv') 
from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace("_Pt-20to40","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=(os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root").replace("_Pt-20to40","")
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_1.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(2590, 3)
(63259, 3)
nCleaned
(22278, 3)
(376, 3)
(19, 3)
GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8 events with 36 fb-1: 27.2762064008
(3, 3)
after cutting in dijet mass:2.52322539606


In [167]:
bkg_3_count_df = rpd.read_root(utils.IO.backgroundName[3],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_3_count_df,utils.IO.bkgProc[3],utils.IO.backgroundName[3])

M_bkg = np.asarray(bkg_3_count_df['diphotonCandidate.M()'][utils.IO.background_df[3].index])
W_bkg = np.asarray(bkg_3_count_df['weight'][utils.IO.background_df[3].index])
M_dijet_bkg = np.asarray(bkg_3_count_df['dijetCandidate.M()'][utils.IO.background_df[3].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_3>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[3].split("/")[len(utils.IO.backgroundName[3].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_3.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(63320, 3)
(56382, 3)
(9397, 3)
(7683, 3)
GluGluHToGG_M-125_13TeV_powheg_pythia8 events with 36 fb-1: 27.9966198953
(2232, 3)
after cutting in dijet mass:8.15203743548


In [168]:
bkg_4_count_df = rpd.read_root(utils.IO.backgroundName[4],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_4_count_df,utils.IO.bkgProc[4],utils.IO.backgroundName[4])

M_bkg = np.asarray(bkg_4_count_df['diphotonCandidate.M()'][utils.IO.background_df[4].index])
W_bkg = np.asarray(bkg_4_count_df['weight'][utils.IO.background_df[4].index])
M_dijet_bkg = np.asarray(bkg_4_count_df['dijetCandidate.M()'][utils.IO.background_df[4].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_4>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace(".root","").replace("output_","")+"  events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[4].split("/")[len(utils.IO.backgroundName[4].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_4.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(84846, 3)
(76431, 3)
(16338, 3)
(13251, 3)
VBFHToGG_M-125_13TeV_powheg_pythia8  events with 36 fb-1: 4.03144847125
(3704, 3)
after cutting in dijet mass:1.13552129729


In [169]:
bkg_5_count_df = rpd.read_root(utils.IO.backgroundName[5],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_5_count_df,utils.IO.bkgProc[5],utils.IO.backgroundName[5])

M_bkg = np.asarray(bkg_5_count_df['diphotonCandidate.M()'][utils.IO.background_df[5].index])
W_bkg = np.asarray(bkg_5_count_df['weight'][utils.IO.background_df[5].index])
M_dijet_bkg = np.asarray(bkg_5_count_df['dijetCandidate.M()'][utils.IO.background_df[5].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_5>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[5].split("/")[len(utils.IO.backgroundName[5].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_5.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(65684, 3)
(57455, 3)
(17530, 3)
(13895, 3)
VHToGG_M125_13TeV_amcatnloFXFX_madspin_pythia8 events with 36 fb-1: 7.99475488541
(3706, 3)
after cutting in dijet mass:1.94793072498


In [170]:
bkg_6_count_df = rpd.read_root(utils.IO.backgroundName[6],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_6_count_df,utils.IO.bkgProc[6],utils.IO.backgroundName[6])

M_bkg = np.asarray(bkg_6_count_df['diphotonCandidate.M()'][utils.IO.background_df[6].index])
W_bkg = np.asarray(bkg_6_count_df['weight'][utils.IO.background_df[6].index])
M_dijet_bkg = np.asarray(bkg_6_count_df['dijetCandidate.M()'][utils.IO.background_df[6].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_6>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[6].split("/")[len(utils.IO.backgroundName[6].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_6.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(123341, 3)
(106395, 3)
(43729, 3)
(33115, 3)
bbHToGG_M-125_4FS_yb2_13TeV_amcatnlo events with 36 fb-1: 1.90807058642
(11563, 3)
after cutting in dijet mass:0.670939787381


In [171]:
bkg_7_count_df = rpd.read_root(utils.IO.backgroundName[7],"bbggSelectionTree", columns = additionalCut_names)
utils.preprocessing.define_process_weight(bkg_7_count_df,utils.IO.bkgProc[7],utils.IO.backgroundName[7])

M_bkg = np.asarray(bkg_7_count_df['diphotonCandidate.M()'][utils.IO.background_df[7].index])
W_bkg = np.asarray(bkg_7_count_df['weight'][utils.IO.background_df[7].index])
M_dijet_bkg = np.asarray(bkg_7_count_df['dijetCandidate.M()'][utils.IO.background_df[7].index])
nTot = np.column_stack((M_bkg,W_bkg,M_dijet_bkg))



print nTot.shape
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]
print nCleaned.shape
#apply MVA cut at 90% eff
nCleaned_MVA = nCleaned[np.where(Y_pred_bkg_7>MVA_Cut)]
print nCleaned_MVA.shape
#apply massWindow +- 1.5
nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
print nCleaned_massWindow.shape
nEvts = sum(nCleaned_massWindow[:,1])
print utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
#import matplotlib.pyplot as plt
#plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
#plt.show()
nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
print nCleaned_mdijet.shape
nEvts_Cut = sum(nCleaned_mdijet[:,1])
print "after cutting in dijet mass:"+str(nEvts_Cut)

from root_numpy import array2root
outTag = '20170711'
processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","")
v=(np.asarray(nCleaned_mdijet[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned_mdijet[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned_mdijet[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')

processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.backgroundName[7].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
v=(np.asarray(nCleaned[:,0]))
v.dtype = [('Mgg', np.float64)]
array2root(v, processPath, "reducedTree", mode ='recreate')
v=(np.asarray(nCleaned[:,2]))
v.dtype = [('Mjj', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(nCleaned[:,1]))
v.dtype = [('weight', np.float64)]
array2root(v, processPath, "reducedTree", mode ='update')
v=(np.asarray(Y_pred_bkg_7.ravel()))
v.dtype = [('MVAOutput', np.float32)]
array2root(v, processPath, "reducedTree", mode ='update')

(350826, 3)
(268863, 3)
(235205, 3)
(177843, 3)
ttHToGG_M125_13TeV_powheg_pythia8_v2 events with 36 fb-1: 10.2090921103
(71988, 3)
after cutting in dijet mass:4.13521144083


In [176]:
data_count_df = rpd.read_root(utils.IO.dataName[0],"bbggSelectionTree", columns = additionalCut_names)
w = (np.ones_like(utils.IO.data_df[0].index)).astype(np.int8)
data_count_df['weight'] = np.multiply(w,data_count_df['isSignal'])

M_data = np.asarray(data_count_df['diphotonCandidate.M()'][utils.IO.data_df[0].index])
W_data = np.asarray(data_count_df['weight'][utils.IO.data_df[0].index])
M_dijet_data = np.asarray(data_count_df['dijetCandidate.M()'][utils.IO.data_df[0].index])
nTot = np.column_stack((M_data,W_data,M_dijet_data))
print nTot.shape


#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,1]!=0),:][0]

print nCleaned.shape
print Y_pred_data.shape


from root_numpy import array2root
outTag = '20170711'

blind = True
if not blind:
    print Y_pred_data.shape
    nCleanedaned_MVA = nCleaned[np.where(Y_pred_data>MVA_Cut)]
    #apply massWindow +- 1.5
    nCleaned_massWindowDown = nCleaned_MVA[np.where(nCleaned_MVA[:,0]>123.5)]
    nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]<126.5)]
    print nCleaned_massWindow.shape
    nEvts = sum(nCleaned_massWindow[:,1])
    print utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace(".root","").replace("output_","")+" events with 36 fb-1: "+str(nEvts)
    Histo_B = np.histogram(nCleaned_massWindow[:,2],bins=30,range=(100,180),weights=nCleaned_massWindow[:,1])
    #import matplotlib.pyplot as plt
    #plt.hist(nCleaned_massWindow[:,2], bins=range(int(min(nCleaned_massWindow[:,2])), int(max(nCleaned_massWindow[:,2])) + 5, 5))
    #plt.show()
    nCleaned_mdijet_down = nCleaned_massWindow[np.where(nCleaned_massWindow[:,2]>100)]
    nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]<140)]
    print nCleaned_mdijet.shape
    nEvts_Cut = sum(nCleaned_mdijet[:,1])
    print "after cutting in dijet mass:"+str(nEvts_Cut)
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","")
    v=(np.asarray(nCleaned_mdijet[:,0]))
    v.dtype = [('Mgg', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='recreate')
    v=(np.asarray(nCleaned_mdijet[:,2]))
    v.dtype = [('Mjj', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(nCleaned_mdijet[:,1]))
    v.dtype = [('weight', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    v=(np.asarray(nCleaned[:,0]))
    v.dtype = [('Mgg', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='recreate')
    v=(np.asarray(nCleaned[:,2]))
    v.dtype = [('Mjj', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(nCleaned[:,1]))
    v.dtype = [('weight', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(Y_pred_data.ravel()))
    v.dtype = [('MVAOutput', np.float32)]
    array2root(v, processPath, "reducedTree", mode ='update')
    
else:
    
    #save preselection data
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    v=(np.asarray(nCleaned[:,0]))
    v.dtype = [('Mgg', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='recreate')
    v=(np.asarray(nCleaned[:,2]))
    v.dtype = [('Mjj', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(nCleaned[:,1]))
    v.dtype = [('weight', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(Y_pred_data.ravel()))
    v.dtype = [('MVAOutput', np.float32)]
    array2root(v, processPath, "reducedTree", mode ='update')
    
    #do blinding
    nCleaned_MVA = nCleaned[np.where(Y_pred_data>MVA_Cut)]
    
    
    nCleaned_massWindow = nCleaned_MVA[np.where(np.logical_or(nCleaned_MVA[:,0]<115,nCleaned_MVA[:,0]>135))]
#    nCleaned_massWindow = nCleaned_massWindowDown[np.where(nCleaned_massWindowDown[:,0]>135)]
    nCleaned_mdijet = nCleaned_massWindow[np.where(np.logical_or(nCleaned_massWindow[:,2]<100,nCleaned_massWindow[:,2]>140))]
#    nCleaned_mdijet = nCleaned_mdijet_down[np.where(nCleaned_mdijet_down[:,2]>140)]
    
    
    processPath=os.path.expanduser('~/HHbbgg_ETH/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_blinded"+".root"
    v=(np.asarray(nCleaned_mdijet[:,0]))
    v.dtype = [('Mgg', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='recreate')
    v=(np.asarray(nCleaned_mdijet[:,2]))
    v.dtype = [('Mjj', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    v=(np.asarray(nCleaned_mdijet[:,1]))
    v.dtype = [('weight', np.float64)]
    array2root(v, processPath, "reducedTree", mode ='update')
    

(248565, 3)
(92390, 3)
(92390,)


In [28]:
reload(utils)
utils.postprocessing.stackFeatures(sig_count_df,additionalCut_names)

in i = 0
(21184,)
(21184, 2)
(21184, 3)
(21184, 4)


In [7]:
print utils.IO.signalName

[]
